In [1]:
import random
import sys
import pygame
from pygame.locals import *
import numpy as np
import matplotlib.pyplot as plt

pygame 2.1.3 (SDL 2.0.22, Python 3.8.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
WIDTH = 280
HEIGHT = 511

BASEY = HEIGHT * 0.8
IMAGES = {}
pygame.font.init()
WINDOW = pygame.display.set_mode((WIDTH, HEIGHT))
Font = pygame.font.SysFont("comicsans", 20)
BIRD = 'bird1.png'
BG = 'bg.png'
PIPE = 'pipe.png'
max_scores = []

Q = np.zeros((7, 21, 2), dtype=float)
FPS = 1000000
FPSCLOCK = pygame.time.Clock()

In [3]:
def static():
    birdxpos = int(WIDTH / 5)
    birdypos = int((HEIGHT - IMAGES['bird'].get_height()) / 2)
    basex = 0
    while (True):
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.display.quit()
                pygame.quit()
                sys.exit()

            elif event.type == KEYDOWN and (event.key == K_SPACE):
                return
            else:
                WINDOW.blit(IMAGES['background'], (0, 0))
                WINDOW.blit(IMAGES['bird'], (birdxpos, birdypos))
                WINDOW.blit(IMAGES['ground'], (basex, BASEY))
                pygame.display.update()
                FPSCLOCK.tick(FPS)

In [4]:
def Collision(birdxpos, birdypos, up_pipes, bttm_pipes):
    if (birdypos >= BASEY - IMAGES['bird'].get_height() or birdypos < 0):
        return True
    for pipe in up_pipes:
        pipeHeight = IMAGES['pipe'][0].get_height()
        if (birdypos < pipeHeight + pipe['y'] and abs(birdxpos - pipe['x']) < IMAGES['pipe'][0].get_width()):
            return True

    for pipe in bttm_pipes:
        if (birdypos + IMAGES['bird'].get_height() > pipe['y'] and abs(birdxpos - pipe['x']) < IMAGES['pipe'][
            0].get_width()):
            return True
    return False

In [5]:
def get_new_pipe():
    pipeHeight = IMAGES['pipe'][1].get_height()
    gap = int(HEIGHT / 4)
    y2 = int(gap + random.randrange(0, int(HEIGHT - IMAGES['ground'].get_height() - 1.2 * gap)))
    pipex = int(WIDTH + 300)
    y1 = int(pipeHeight - y2 + gap)

    pipe = [
        {'x': pipex, 'y': -y1},
        {'x': pipex, 'y': y2}
    ]
    return pipe


In [6]:
def ai_play(x, y):
    jump = False

    if (Q[x][y][1] > Q[x][y][0]):
        jump = True

    return jump

In [7]:
def convert(birdxpos, birdypos, bttm_pipes):
    x = min(280, bttm_pipes[0]['x'])
    y = bttm_pipes[0]['y'] - birdypos
    if (y < 0):
        y = abs(y) + 500
    return int(x / 40 - 1), int(y / 40)



In [8]:
def Q_update(x_prev, y_prev, jump, reward, x_new, y_new):
    if jump:
        Q[x_prev][y_prev][1] = 0.6 * Q[x_prev][y_prev][1] + 0.4 * (
                    reward + max(Q[x_new][y_new][0], Q[x_new][y_new][1]))
    else:
        Q[x_prev][y_prev][0] = 0.6 * Q[x_prev][y_prev][0] + 0.4 * (
                    reward + max(Q[x_new][y_new][0], Q[x_new][y_new][1]))


In [9]:
def game_start(generation, x, y):
    score = 0
    birdxpos = int(WIDTH / 5)
    birdypos = int(HEIGHT / 2)
    basex1 = 0
    basex2 = WIDTH

    bgx1 = 0
    bgx2 = IMAGES['background'].get_width()

    newPipe1 = get_new_pipe()
    newPipe2 = get_new_pipe()

    up_pipes = [
        {'x': WIDTH + 200, 'y': newPipe1[0]['y']},
        {'x': WIDTH + 500, 'y': newPipe2[0]['y']}
    ]

    bttm_pipes = [
        {'x': WIDTH + 200, 'y': newPipe1[1]['y']},
        {'x': WIDTH + 500, 'y': newPipe2[1]['y']}
    ]

    pipeVelx = -4

    birdyvel = -9
    birdymaxvel = 10
    birdyvelmin = -8
    birdyacc = 1

    playerFlapAccv = -8
    playerFlapped = False

    while (True):

        x_prev, y_prev = convert(birdxpos, birdypos, bttm_pipes)
        jump = ai_play(x_prev, y_prev)

        for event in pygame.event.get():
            if event.type == QUIT:
                #plt.scatter(x, y)

                pygame.quit()
                sys.exit()

        if jump:
            if birdypos > 0:
                birdyvel = playerFlapAccv
                playerFlapped = True

        playerMidPos = birdxpos + IMAGES['bird'].get_width() / 2
        for pipe in up_pipes:
            pipeMidPos = pipe['x'] + IMAGES['pipe'][0].get_width() / 2
            if pipeMidPos <= playerMidPos < pipeMidPos + 4:
                score += 1

        if birdyvel < birdymaxvel and not playerFlapped:
            birdyvel += birdyacc

        if playerFlapped:
            playerFlapped = False

        playerHeight = IMAGES['bird'].get_height()

        birdypos = birdypos + min(birdyvel, BASEY - birdypos - playerHeight)

        for upperPipe, lowerPipe in zip(up_pipes, bttm_pipes):
            upperPipe['x'] += pipeVelx
            lowerPipe['x'] += pipeVelx

        if (0 < up_pipes[0]['x'] < 5):
            newPipe = get_new_pipe()
            up_pipes.append(newPipe[0])
            bttm_pipes.append(newPipe[1])

        if (up_pipes[0]['x'] < -IMAGES['pipe'][0].get_width()):
            up_pipes.pop(0)
            bttm_pipes.pop(0)
        basex1 -= 4
        basex2 -= 4
        if (basex1 <= -IMAGES['ground'].get_width()):
            basex1 = basex2
            basex2 = basex1 + IMAGES['ground'].get_width()

        bgx1 -= 2
        bgx2 -= 2
        if (bgx1 <= -IMAGES['background'].get_width()):
            bgx1 = bgx2
            bgx2 = bgx1 + IMAGES['background'].get_width()
        dead = Collision(birdxpos, birdypos, up_pipes, bttm_pipes)
        x_new, y_new = convert(birdxpos, birdypos, bttm_pipes)
        if dead:
            reward = -1000
            Q_update(x_prev, y_prev, jump, reward, x_new, y_new)
            max_scores.append(score)
            return score

        reward = 1

        Q_update(x_prev, y_prev, jump, reward, x_new, y_new)

        WINDOW.blit(IMAGES['background'], (bgx1, 0))
        WINDOW.blit(IMAGES['background'], (bgx2, 0))
        for upperPipe, lowerPipe in zip(up_pipes, bttm_pipes):
            WINDOW.blit(IMAGES['pipe'][0], (upperPipe['x'], upperPipe['y']))
            WINDOW.blit(IMAGES['pipe'][1], (lowerPipe['x'], lowerPipe['y']))
        WINDOW.blit(IMAGES['ground'], (basex1, BASEY))
        WINDOW.blit(IMAGES['ground'], (basex2, BASEY))
        text1 = Font.render("Score: " + str(score), 1, (255, 255, 255))
        text2 = Font.render("Generation: " + str(generation), 1, (255, 255, 255))
        WINDOW.blit(text1, (WIDTH - 10 - text1.get_width(), 10))
        WINDOW.blit(text2, (0, 0))
        WINDOW.blit(IMAGES['bird'], (birdxpos, birdypos))

        pygame.display.update()
        FPSCLOCK.tick(FPS)

In [10]:
pygame.init()

IMAGES['ground'] = pygame.image.load('ground.png').convert_alpha()
IMAGES['pipe'] = (
pygame.transform.rotate(pygame.image.load(PIPE).convert_alpha(), 180), pygame.image.load(PIPE).convert_alpha())
IMAGES['background'] = pygame.image.load(BG).convert()
IMAGES['bird'] = pygame.image.load(BIRD).convert_alpha()
generation = 1
gens = 100
static()
x = []
y = []

for i in range(gens):
    score = game_start(generation, x, y)
    if (score == -1):
        break
    x.append(generation)
    y.append(score)
    generation += 1
#plt.plot(max_scores, range(gens))

SystemExit: 

D:\Agentni3\lib\site-packages\IPython\core\interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
plt.xlabel("GENERATION")
plt.ylabel("SCORE")
plt.plot(x,y)
z = np.polyfit(x, y, 2)
p = np.poly1d(z)

#add trendline to plot
plt.plot(x, p(x))# Plot some data on the axes.
print(x)
print(y)
print(generation)
print(np.average(y))